<a href="https://colab.research.google.com/github/manvgom/IML/blob/main/BigML_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bigml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 4.2 MB/s 
     |████████████████████████████████| 235 kB 63.2 MB/s 
     |████████████████████████████████| 54 kB 2.2 MB/s 
  Created wheel for bigml: filename=bigml-7.4.2-py3-none-any.whl size=449826 sha256=0789eda9ca08aa4704a06ad97c9847f71df0c085d6dce175b7b5d8db84acbe7c
  Stored in directory: /root/.cache/pip/wheels/a4/ea/23/2c845e82f98727fd435d5381658ee02b85deea7ecd42067167
  Created wheel for bigml-chronos: filename=bigml_chronos-1.0.0-py3-none-any.whl size=14801 sha256=5515c54ecc2b6af1ad64d3b04d305519c358c37df6483bf8bd970af7602cc275
  Stored in directory: /root/.cache/pip/wheels/66/7e/e2/3e92cb7534923c57674826fd6574b0d1e44d4eb62a7df1459b
Successfully built bigml bigml-chronos


In [3]:
from bigml.api import BigML

In [4]:
api = BigML ('manvgom','6e063c3663f309b6034bc638f2eb9b333ca8e7c1')

In [5]:
project = api.create_project({'name':'220601_Test'})
#project

In [6]:
api = BigML('manvgom','6e063c3663f309b6034bc638f2eb9b333ca8e7c1', project = project['resource'])

In [7]:
source = api.create_source('https://cleverdata.io/csv/telecom.csv')
api.ok(source)

True

In [8]:
dataset = api.create_dataset(source)
api.ok(dataset)

True

In [9]:
train_dataset_ori = api.create_dataset(dataset,{"name":"telecom 80%","sample_rate":0.8,"seed":"mi semilla"})
test_dataset_ori = api.create_dataset(dataset,{"name":"telecom 20%","sample_rate":0.8,"seed":"mi semilla","out_of_bag": True})
api.ok(train_dataset_ori)
api.ok(test_dataset_ori)

True

In [13]:
def create_eval_models(train_ds,test_ds):
  tree = api.create_model(train_ds)
  api.ok(tree)
  tree_eval = api.create_evaluation(tree,test_ds)

  forest = api.create_ensemble(train_ds)
  api.ok(forest)
  forest_eval = api.create_evaluation(forest,test_ds)

  cnn = api.create_deepnet(train_ds)
  api.ok(cnn)
  cnn_eval = api.create_evaluation(cnn,test_ds)

In [14]:
create_eval_models(train_dataset_ori, test_dataset_ori)

In [15]:
anomaly_detector = api.create_anomaly(train_dataset_ori, {"excluded_fields":["Churn"]})
api.ok(anomaly_detector)

True

In [16]:
batch_anomaly_train = api.create_batch_anomaly_score(anomaly_detector, train_dataset_ori, {"all_fields": True, "output_dataset": True})
api.ok(batch_anomaly_train)

batch_anomaly_test = api.create_batch_anomaly_score(anomaly_detector, test_dataset_ori, {"all_fields": True, "output_dataset": True})
api.ok(batch_anomaly_test)

True

In [19]:
score_ds = batch_anomaly_train['object']['output_dataset_resource']
change_target = api.update_dataset(score_ds, {"objective_field":{"id":"000013"}})

In [20]:
create_eval_models(batch_anomaly_train['object']['output_dataset_resource'], 
                           batch_anomaly_test['object']['output_dataset_resource'])